<a href="https://colab.research.google.com/github/okolenchuk/DS_HW_NLP/blob/main/task_2_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyLDAvis

In [2]:
import os, re
import pandas as pd
import numpy as np
import math
import string

from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import *

from collections import Counter, defaultdict

import pyLDAvis
import pyLDAvis.gensim_models

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

import warnings

%matplotlib inline
warnings.filterwarnings("ignore")

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable
/usr/local/lib/python3.7/dist-packages/past/builtins/misc.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping


In [3]:
from google.colab import drive
drive.mount('/content/drive')

data_path = '/content/drive/MyDrive/bbc'

Mounted at /content/drive


In [4]:
directory = []
file = []
title = []
text = []
label = []
 
for dirname, _ , filenames in os.walk(data_path):
    for filename in filenames:
        directory.append(dirname)
        file.append(filename)
        label.append(dirname.split('/')[-1])
        fullpathfile = os.path.join(dirname,filename)
        with open(fullpathfile, 'r', encoding="utf8", errors='ignore') as infile:
            intext = ''
            firstline = True
            for line in infile:
                if firstline:
                    title.append(line.replace('\n',''))
                    firstline = False
                else:
                    intext = intext + ' ' + line.replace('\n','')
            text.append(intext)

In [5]:
DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

fulldf = pd.DataFrame(list(zip(directory, file, title, text, label)), 
               columns =['directory', 'file', 'title', 'text', 'label'])

df = fulldf.filter(['text','label'], axis=1)
df.head()

,text,label
0,Uganda's authorities have banned the play Th...,entertainment
1,The plot of an international bestseller that...,entertainment
2,The 10 authors shortlisted for a Richard and...,entertainment
3,Hollywood film star Liam Neeson has held a d...,entertainment
4,"Young book fans have voted Fergus Crane, a s...",entertainment


In [7]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [15]:
import spacy
lemmatizer = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""


def lemmatize(text, lemmatizer=lemmatizer):
    try:
        sent = lemmatizer(text)
        return " ".join([word.lemma_ for word in sent])
    except:
        return " "

In [17]:
df.text = df.text.apply(words_only)
df.text = df.text.apply(lemmatize)
df.head()

,text,label
0,Uganda s authority have ban the play the Vagin...,entertainment
1,the plot of an international bestseller that t...,entertainment
2,the author shortlist for a Richard and Judy bo...,entertainment
3,Hollywood film star Liam Neeson ha hold a dinn...,entertainment
4,young book fan have vote Fergus Crane a story ...,entertainment


In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [20]:
from nltk.corpus import stopwords
stop_words= stopwords.words('english') + list(string.punctuation)

In [21]:
tokenizer = nltk.WordPunctTokenizer()
def preprocess(sent, tokenizer=tokenizer):
    tokens = [word for word in tokenizer.tokenize(sent.lower()) if (word not in stop_words) & (len(word)>3)]
    return tokens

df['clear_text'] = df.text.apply(preprocess)   
df.head()

,text,label,clear_text
0,Uganda s authority have ban the play the Vagin...,entertainment,"[uganda, authority, play, vagina, monologues, ..."
1,the plot of an international bestseller that t...,entertainment,"[plot, international, bestseller, thousand, re..."
2,the author shortlist for a Richard and Judy bo...,entertainment,"[author, shortlist, richard, judy, book, award..."
3,Hollywood film star Liam Neeson ha hold a dinn...,entertainment,"[hollywood, film, star, liam, neeson, hold, di..."
4,young book fan have vote Fergus Crane a story ...,entertainment,"[young, book, vote, fergus, crane, story, take..."


In [22]:
labels = df.label.unique()
df.label.value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    392
Name: label, dtype: int64

In [23]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [24]:
sport = df[df['label']=='sport'].clear_text.tolist()
# create dictionary and corpus
# create dictionary
id2word_sport=corpora.Dictionary(sport)
#create corpus
corpus_sport=[id2word_sport.doc2bow(text) for text in sport]

# Build LDA model
lda_model_sport= gensim.models.ldamodel.LdaModel(corpus=corpus_sport,id2word=id2word_sport,num_topics=5, update_every=1,
                                                 chunksize=100,passes=10,alpha='auto',per_word_topics=True)


lda_model_sport.print_topics()

# coherence score
coherence_model_lda=CoherenceModel(model=lda_model_sport,texts=sport,dictionary=id2word_sport,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model_sport,corpus_sport,id2word_sport)
vis


Coherence Score: 0.48449850952231016


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.110279 -0.132123       1        1  35.803515
0      0.102402 -0.086545       2        1  26.871773
1      0.157903  0.194173       3        1  21.396269
2     -0.158721 -0.038352       4        1  10.614419
4     -0.211862  0.062847       5        1   5.314023, topic_info=          Term        Freq       Total Category  logprob  loglift
1725   england  409.000000  409.000000  Default  30.0000  30.0000
481      match  389.000000  389.000000  Default  29.0000  29.0000
1570      open  242.000000  242.000000  Default  28.0000  28.0000
67        year  699.000000  699.000000  Default  27.0000  27.0000
4518      seed  299.000000  299.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
270   champion   26.449840  145.829941   Topic5  -5.0756   1.2276
241       make   27.163423  441.151038   Topic5  -5.0490   0.1473
1         also   24.915135  301.770094   Topic5  -5.1354   0.4406
376    problem   19.715027   86.082301   Topic5  -5.3695   1.4609
33        like   20.334555  244.512263   Topic5  -5.3386   0.4479

[349 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1927      2  0.984145  advantage
7067      4  0.991134     agassi
922       1  0.957748      agree
1         1  0.546774       also
1         2  0.096100       also
...     ...       ...        ...
67        3  0.061441       year
67        4  0.172892       year
67        5  0.011431       year
5898      3  0.985905    zealand
7705      5  0.970530  zvonareva

[569 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 2, 3, 5])

Для новостной категории спорт наиболее популярные слова: 
1. Страны и территории (England, Ireland, Wales)
2. Спортивные игры и все что с ними связано (play, match, game)

Из этого делаем вывод, что в данной категории в основном освещаются игры/матчи между разными командами и странами и их исход.

In [25]:
business = df[df['label']=='business'].clear_text.tolist()
# create dictionary and corpus
# create dictionary
id2word_business=corpora.Dictionary(business)
#create corpus
corpus_business=[id2word_business.doc2bow(text) for text in business]

# Build LDA model
lda_model_business= gensim.models.ldamodel.LdaModel(corpus=corpus_business,id2word=id2word_business,num_topics=5, update_every=1,
                                                    chunksize=100,passes=10,alpha='auto',per_word_topics=True)


lda_model_business.print_topics()

# coherence score
coherence_model_lda=CoherenceModel(model=lda_model_business,texts=business,dictionary=id2word_business,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model_business,corpus_business,id2word_business)
vis


Coherence Score: 0.3871388072084154


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.151324  0.141443       1        1  29.410171
0     -0.076496 -0.071911       2        1  23.059583
2      0.119866 -0.183001       3        1  20.405542
1      0.041873  0.078560       4        1  20.003441
4     -0.236568  0.034910       5        1   7.121263, topic_info=          Term        Freq       Total Category  logprob  loglift
18       china  630.000000  630.000000  Default  30.0000  30.0000
441    economy  300.000000  300.000000  Default  29.0000  29.0000
161    company  661.000000  661.000000  Default  28.0000  28.0000
197     growth  334.000000  334.000000  Default  27.0000  27.0000
98        rate  255.000000  255.000000  Default  26.0000  26.0000
..         ...         ...         ...      ...      ...      ...
599       give   27.502127  105.170791   Topic5  -5.3707   1.3008
183  financial   26.248960  150.304393   Topic5  -5.4174   0.8970
867      share   27.741180  315.702950   Topic5  -5.3621   0.2102
159      chief   25.611743  264.810772   Topic5  -5.4419   0.3061
26       could   25.245079  250.201791   Topic5  -5.4564   0.3484

[351 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
2043      2  0.064050       accounting
2043      3  0.426998       accounting
2043      5  0.491048       accounting
938       1  0.021898           accuse
938       2  0.394161           accuse
...     ...       ...              ...
139       5  0.008477             york
338       2  0.983521          yugansk
3151      2  0.972563  yuganskneftegas
339       2  0.996251            yukos
4125      3  0.943803       yushchenko

[599 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 1, 3, 2, 5])

Для категории бизнес наиболее популярными являются слова: China, economy, company, growth, rate.

In [26]:
politics = df[df['label']=='politics'].clear_text.tolist()
# create dictionary and corpus
# create dictionary
id2word_politics=corpora.Dictionary(politics)
#create corpus
corpus_politics=[id2word_politics.doc2bow(text) for text in politics]

# Build LDA model
lda_model_politics= gensim.models.ldamodel.LdaModel(corpus=corpus_politics,id2word=id2word_politics,num_topics=5, update_every=1,
                                                    chunksize=100,passes=10,alpha='auto',per_word_topics=True)


lda_model_politics.print_topics()

# coherence score
coherence_model_lda=CoherenceModel(model=lda_model_politics,texts=politics,dictionary=id2word_politics,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model_politics,corpus_politics,id2word_politics)
vis


Coherence Score: 0.32452440416024575


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.156359 -0.004739       1        1  44.788213
3      0.091235 -0.025842       2        1  32.085498
0      0.105195  0.043167       3        1  13.060778
1     -0.167096 -0.203226       4        1   6.057522
2     -0.185693  0.190640       5        1   4.007989, topic_info=             Term        Freq       Total Category  logprob  loglift
129        labour  769.000000  769.000000  Default  30.0000  30.0000
103      election  725.000000  725.000000  Default  29.0000  29.0000
137          lord  221.000000  221.000000  Default  28.0000  28.0000
152         party  795.000000  795.000000  Default  27.0000  27.0000
345    chancellor  245.000000  245.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
508         think   15.551449  182.811733   Topic5  -5.4341   0.7526
387       include   13.128563  109.385520   Topic5  -5.6034   1.0968
467   association   12.516861   82.080503   Topic5  -5.6511   1.3363
1041        child   12.350320  128.542461   Topic5  -5.6645   0.8743
459         woman   12.010879   72.414787   Topic5  -5.6924   1.4203

[362 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
6201      3  0.914778  abortion
7725      5  0.953415   addison
516       1  0.197970     adult
516       3  0.791882     adult
7726      5  0.959854  adultery
...     ...       ...       ...
323       1  0.441371      year
323       2  0.263936      year
323       3  0.239538      year
323       4  0.039923      year
323       5  0.015526      year

[654 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 4, 1, 2, 3])

Для категории политики: labour, election, lord, party, chancellor. 
Большая часть тем так или иначе связана с выборами.

In [27]:
tech = df[df['label']=='tech'].clear_text.tolist()
# create dictionary and corpus
# create dictionary
id2word_tech=corpora.Dictionary(tech)
#create corpus
corpus_tech=[id2word_tech.doc2bow(text) for text in tech]

# Build LDA model
lda_model_tech= gensim.models.ldamodel.LdaModel(corpus=corpus_tech,id2word=id2word_tech,num_topics=5, update_every=1,
                                                    chunksize=100,passes=10,alpha='auto',per_word_topics=True)


lda_model_tech.print_topics()

# coherence score
coherence_model_lda=CoherenceModel(model=lda_model_tech,texts=tech,dictionary=id2word_tech,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model_tech,corpus_tech,id2word_tech)
vis


Coherence Score: 0.4110898192571014


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.068116 -0.057540       1        1  35.998044
1      0.017786 -0.151109       2        1  23.021362
2      0.139290  0.144839       3        1  19.604559
4      0.117967  0.018062       4        1  17.362821
0     -0.343159  0.045747       5        1   4.013215, topic_info=       Term        Freq       Total Category  logprob  loglift
395    game  357.000000  357.000000  Default  30.0000  30.0000
201  mobile  613.000000  613.000000  Default  29.0000  29.0000
212   phone  536.000000  536.000000  Default  28.0000  28.0000
405   music  362.000000  362.000000  Default  27.0000  27.0000
801  player  287.000000  287.000000  Default  26.0000  26.0000
..      ...         ...         ...      ...      ...      ...
256   world   21.101128  280.730625   Topic5  -5.2127   0.6275
257    year   21.155574  458.393617   Topic5  -5.2101   0.1398
519    like   20.996156  320.548090   Topic5  -5.2177   0.4899
136   would   20.955772  427.347159   Topic5  -5.2196   0.2004
914    late   20.769925  114.029003   Topic5  -5.2285   1.5126

[361 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
1110      1  0.772151  access
1110      3  0.172249  access
1110      4  0.053457  access
420       1  0.606447  accord
420       2  0.170231  accord
...     ...       ...     ...
257       2  0.253058    year
257       3  0.104713    year
257       4  0.150526    year
257       5  0.045812    year
1108      1  0.986589   young

[638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 3, 5, 1])

Для категории tech: game, mobile, phone, music, player.
Заметное преобладание тематики компьютерных игр. 

In [28]:
entertainment = df[df['label']=='entertainment'].clear_text.tolist()
# create dictionary and corpus
# create dictionary
id2word_entertainment=corpora.Dictionary(entertainment)
#create corpus
corpus_entertainment=[id2word_entertainment.doc2bow(text) for text in entertainment]

# Build LDA model
lda_model_entertainment= gensim.models.ldamodel.LdaModel(corpus=corpus_entertainment,id2word=id2word_entertainment,num_topics=5, update_every=1,
                                                    chunksize=100,passes=10,alpha='auto',per_word_topics=True)


lda_model_entertainment.print_topics()

# coherence score
coherence_model_lda=CoherenceModel(model=lda_model_entertainment,texts=entertainment,dictionary=id2word_entertainment,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print('\nCoherence Score:',coherence_lda)


pyLDAvis.enable_notebook()
vis=pyLDAvis.gensim_models.prepare(lda_model_entertainment,corpus_entertainment,id2word_entertainment)
vis


Coherence Score: 0.3855551352137768


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.086016 -0.167168       1        1  28.140478
0      0.157734  0.057659       2        1  22.620415
2     -0.166756 -0.062659       3        1  21.480912
4      0.010185  0.101114       4        1  14.415015
3     -0.087179  0.071053       5        1  13.343181, topic_info=       Term        Freq       Total Category  logprob  loglift
157    film  865.000000  865.000000  Default  30.0000  30.0000
1515  music  400.000000  400.000000  Default  29.0000  29.0000
159    good  581.000000  581.000000  Default  28.0000  28.0000
3474   band  210.000000  210.000000  Default  27.0000  27.0000
3237   song  208.000000  208.000000  Default  26.0000  26.0000
...     ...         ...         ...      ...      ...      ...
189    make   33.818138  321.366652   Topic5  -5.5231  -0.2374
644   first   30.686145  236.923747   Topic5  -5.6203  -0.0298
111    work   27.963267  151.671096   Topic5  -5.7132   0.3233
223    time   28.013042  193.321892   Topic5  -5.7115   0.0825
297    well   25.340243  180.509103   Topic5  -5.8117   0.0508

[379 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
998       1  0.044203  academy
998       3  0.884051  academy
998       4  0.029468  academy
998       5  0.044203  academy
114       1  0.186373   accord
...     ...       ...      ...
234       1  0.390880     year
234       2  0.123611     year
234       3  0.183747     year
234       4  0.183747     year
234       5  0.116930     year

[670 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 3, 5, 4])

Для категории entertainment: film, music, good, band, song. 
Явное выделение категорий произведений искусства (музыки, песен, кино) и их оценка.